<a href="https://colab.research.google.com/github/Clemenciah/Personal-Learning/blob/master/WordEmbedding_keras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#!pip install tensorflow==2.0.0-beta0

In [2]:
!pip install tensorflow==1.0.0

### Custom Word Embeddings

Keras Embeding Layer is used to learn custom word embeddings.
We will perform simple text classification tasks that will use word embeddings. Here we learn word embedding which we use for text classification

In [0]:
import numpy
import tensorflow as tf
#from tensorflow.keras import datasets, layers, models#
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

In [0]:
from keras import layers

Define our dataset. We'll use a custom dataset containing movie reviews

In [0]:
corpus = [
    # Positive Reviews

    'This is an excellent movie',
    'The move was fantastic I like it',
    'You should watch it is brilliant',
    'Exceptionally good',
    'Wonderfully directed and executed I like it',
    'Its a fantastic series',
    'Never watched such a brillent movie',
    'It is a Wonderful movie',

    # Negtive Reviews

    "horrible acting",
    'waste of money',
    'pathetic picture',
    'It was very boring',
    'I did not like the movie',
    'The movie was horrible',
    'I will not recommend',
    'The acting is pathetic'
]

The corpus has 8 positive reviews and 8 negative reviews. Next we create a label set for our data

In [0]:
sentiments = array([1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0])

The first parameter to the Embedding() layer is the vocabulary, or number of unique words in the corpus. Let's first find the total number of words in our corpus:

In [9]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
from nltk.tokenize import word_tokenize

all_words = []
for sentence in corpus:
  tokenize_word = word_tokenize(sentence)
  for word in tokenize_word:
    all_words.append(word)

In [26]:

len(all_words)

71

We simply iterate through each sentence in our corpus and then tokenize the sentence into words. Next, we iterate through the list of all the words and append the words into the all_words list. 

all_words list above contains all words including duplicates. We will retrieve unique words from the list by passing the list into a set function.

In [27]:
unique_words = set(all_words)
len(unique_words)

45

We will add a buffer of 5 to our vocabulary size and will set the value of vocab_length to 50.

The Embedding layer expects the words to be in numeric form. Therefore, we need to convert the sentences in our corpus to numbers. One way to convert text to numbers is by using the one_hot function from the keras.preprocessing.text library. The function takes sentence and the total length of the vocabulary and returns the sentence in numeric form.

In [28]:
vocab_length = 50

embedded_sentences = [one_hot(sentence,vocab_length) for sentence in corpus ]
embedded_sentences

[[1, 28, 27, 20, 27],
 [40, 41, 48, 49, 42, 40, 23],
 [29, 13, 32, 23, 28, 14],
 [32, 36],
 [31, 5, 7, 4, 42, 40, 23],
 [7, 43, 49, 32],
 [30, 47, 24, 43, 42, 27],
 [23, 28, 43, 45, 27],
 [9, 42],
 [19, 35, 14],
 [26, 6],
 [23, 48, 18, 10],
 [42, 43, 30, 40, 40, 27],
 [40, 27, 48, 9],
 [42, 40, 30, 22],
 [40, 42, 28, 26]]

You can see that our first sentence contained five words, therefore we have five integers in the first list item

The embedding layer expects sentences to be of equal size. However, our encoded sentences are of different sizes. One way to make all the sentences of uniform size is to increase the lenght of all the sentences and make it equal to the length of the largest sentence. Let's first find the largest sentence in our corpus and then we will increase the length of all the sentences to the length of the largest sentence.

In [29]:
word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(corpus, key=word_count)
length_long_sentence = len(word_tokenize(longest_sentence))
length_long_sentence

7

Lambda functions are unanonymous functions. They operate normally like other functions. e.g 

```
# remainder = lambda num: num % 2

print(remainder(5))
```



We use a lambda expression to find the length of all the sentences. We then use the max function to return the longest sentence. Finally the longest sentence is tokenized into words and the number of words are counted using the len function.

Next to make all the sentences of equal size, we will add zeros to the empty indexes that will be created as a result of increasing the sentence length. To append the zeros at the end of the sentencses, we can use the pad_sequences method. The first parameter is the list of encoded sentences of unequal sizes, the second parameter is the size of the longest sentence or the padding index, while the last parameter is padding where you specify post to add padding at the end of sentences.

In [30]:
padded_sentence = pad_sequences(embedded_sentences, length_long_sentence, padding='post')
padded_sentence

array([[ 1, 28, 27, 20, 27,  0,  0],
       [40, 41, 48, 49, 42, 40, 23],
       [29, 13, 32, 23, 28, 14,  0],
       [32, 36,  0,  0,  0,  0,  0],
       [31,  5,  7,  4, 42, 40, 23],
       [ 7, 43, 49, 32,  0,  0,  0],
       [30, 47, 24, 43, 42, 27,  0],
       [23, 28, 43, 45, 27,  0,  0],
       [ 9, 42,  0,  0,  0,  0,  0],
       [19, 35, 14,  0,  0,  0,  0],
       [26,  6,  0,  0,  0,  0,  0],
       [23, 48, 18, 10,  0,  0,  0],
       [42, 43, 30, 40, 40, 27,  0],
       [40, 27, 48,  9,  0,  0,  0],
       [42, 40, 30, 22,  0,  0,  0],
       [40, 42, 28, 26,  0,  0,  0]], dtype=int32)

Now all sentences are the same length. We will create a sentiment classification model using word embeddings. The model will have an embedding layer and no hidden layers

In [0]:
model =  Sequential()
model.add(Embedding(vocab_length, 20, input_length=length_long_sentence))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

Above, we created a Sequential model and added the Embedding layer as the first layer to the model. The length of the vocabulary is specified by the vocab_length parameter. The dimension of each word vector will be 20 and the input_length will be the length of the longest sentence, which is 7. Next, the Embedding layer is flattened so that it can be directly used with the densely connected layer. Since it is a binary classification problem, we use the sigmoid function as the loss function at the dense layer.

In [32]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 7, 20)             1000      
_________________________________________________________________
flatten_2 (Flatten)          (None, 140)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 141       
Total params: 1,141
Trainable params: 1,141
Non-trainable params: 0
_________________________________________________________________
None


The first layer has 1000 trainable parameters. This is because our vocabulary size is 50 and each word will be presented as a 20 dimensional vector. Hence the total number of trainable parameters will be 1000. Similarly, the output from the embedding layer will be a sentence with 7 words where each word is represented by a 20 dimensional vector. However, when the 2D output is flattened, we get a 140 dimensional vector (7 x 20). The flattened vector is directly connected to the dense layer that contains 1 neuran.

In [33]:
model.fit(padded_sentence, sentiments, epochs = 100, verbose=1)

Epoch 1/100
16/16 [==============================] - 0s 12ms/step - loss: 0.6889 - acc: 0.6250
Epoch 2/100
16/16 [==============================] - 0s 110us/step - loss: 0.6853 - acc: 0.6875
Epoch 3/100
16/16 [==============================] - 0s 138us/step - loss: 0.6817 - acc: 0.8125
Epoch 4/100
16/16 [==============================] - 0s 130us/step - loss: 0.6781 - acc: 0.8750
Epoch 5/100
16/16 [==============================] - 0s 106us/step - loss: 0.6745 - acc: 0.8750
Epoch 6/100
16/16 [==============================] - 0s 112us/step - loss: 0.6709 - acc: 0.8750
Epoch 7/100
16/16 [==============================] - 0s 105us/step - loss: 0.6673 - acc: 0.8750
Epoch 8/100
16/16 [==============================] - 0s 108us/step - loss: 0.6637 - acc: 0.8750
Epoch 9/100
16/16 [==============================] - 0s 101us/step - loss: 0.6600 - acc: 0.8750
Epoch 10/100
16/16 [==============================] - 0s 94us/step - loss: 0.6564 - acc: 0.9375
Epoch 11/100
16/16 [=====================

We train and test our model using the same corpus.

verbose = 1, which includes both progress bar and one line per epoch. verbose = 0, means silent. verbose = 2, one line per epoch i.e. epoch no./total no. of epochs.

In [34]:
loss, accuracy = model.evaluate(padded_sentence, sentiments, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 100.000000


### Loading Pretrained Word embeddings

In [1]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding

Using TensorFlow backend.


In [0]:
corpus = [
    # Positive Reviews

    'This is an excellent movie',
    'The move was fantastic I like it',
    'You should watch it is brilliant',
    'Exceptionally good',
    'Wonderfully directed and executed I like it',
    'Its a fantastic series',
    'Never watched such a brillent movie',
    'It is a Wonderful movie',

    # Negtive Reviews

    "horrible acting",
    'waste of money',
    'pathetic picture',
    'It was very boring',
    'I did not like the movie',
    'The movie was horrible',
    'I will not recommend',
    'The acting is pathetic'
]


In [0]:
sentiments = array([1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0])


*   In the last section we used one_hot function to convert text to vectors. Here we'll use Tokenizer function.
*   You simply have to pass your corpus to the Tokenizer's fit_on_text method.




In [0]:
from keras.preprocessing.text import Tokenizer

In [9]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(corpus)
corpus

['This is an excellent movie',
 'The move was fantastic I like it',
 'You should watch it is brilliant',
 'Exceptionally good',
 'Wonderfully directed and executed I like it',
 'Its a fantastic series',
 'Never watched such a brillent movie',
 'It is a Wonderful movie',
 'horrible acting',
 'waste of money',
 'pathetic picture',
 'It was very boring',
 'I did not like the movie',
 'The movie was horrible',
 'I will not recommend',
 'The acting is pathetic']

To get the number of unique words in the text, you can simply count the length of word_index dictionary of the word_tokenizer object. 1 is added to the vocabulary size in order to store the dimensions for the words for which no pretrained word embeddings exist.

In [7]:
vocab_length = len(word_tokenizer.word_index) + 1
vocab_length

44

Finally, to convert sentences to their numeric counterpart, call the texts_to_sequences function and pass it the whole corpus.

In [10]:
embedded_sentences = word_tokenizer.texts_to_sequences(corpus)
embedded_sentences

[[14, 3, 15, 16, 1],
 [4, 17, 6, 9, 5, 7, 2],
 [18, 19, 20, 2, 3, 21],
 [22, 23],
 [24, 25, 26, 27, 5, 7, 2],
 [28, 8, 9, 29],
 [30, 31, 32, 8, 33, 1],
 [2, 3, 8, 34, 1],
 [10, 11],
 [35, 36, 37],
 [12, 38],
 [2, 6, 39, 40],
 [5, 41, 13, 7, 4, 1],
 [4, 1, 6, 10],
 [5, 42, 13, 43],
 [4, 11, 3, 12]]

The next step is to find the number of words in the longest sentence and then to apply padding to the sentences having shorter lengths than the length of the longest sentence.

In [13]:
from nltk.tokenize import word_tokenize

word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(corpus, key = word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

padded_sentence = pad_sequences(embedded_sentences, length_long_sentence, padding='post')

padded_sentence

array([[14,  3, 15, 16,  1,  0,  0],
       [ 4, 17,  6,  9,  5,  7,  2],
       [18, 19, 20,  2,  3, 21,  0],
       [22, 23,  0,  0,  0,  0,  0],
       [24, 25, 26, 27,  5,  7,  2],
       [28,  8,  9, 29,  0,  0,  0],
       [30, 31, 32,  8, 33,  1,  0],
       [ 2,  3,  8, 34,  1,  0,  0],
       [10, 11,  0,  0,  0,  0,  0],
       [35, 36, 37,  0,  0,  0,  0],
       [12, 38,  0,  0,  0,  0,  0],
       [ 2,  6, 39, 40,  0,  0,  0],
       [ 5, 41, 13,  7,  4,  1,  0],
       [ 4,  1,  6, 10,  0,  0,  0],
       [ 5, 42, 13, 43,  0,  0,  0],
       [ 4, 11,  3, 12,  0,  0,  0]], dtype=int32)

 The next step is to load the GloVe word embeddings and then create our embedding matrix that contains the words in our corpus and their corresponding values from GloVe embeddings. 

In [0]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary =dict()
glove_file = open('/content/drive/My Drive/glove.6B.100d.txt', encoding='utf8')

In [13]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


This file contains 100 dimensional word embeddings. We also created an empty dictionary that will store our word embeddings.

We will create a dictionary that will contain words as keys and the corresponding 100 dimensional vectors as values, in the form of an array.

In [0]:
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions

glove_file.close()

The dictionary embeddings_dictionary 
now contains words and corresponding GloVe embeddings for all the words.


---
We want the word embeddings for only those words that are present in our corpus. We will create a two dimensional numpy array of 44 (size of vocabulary) rows and 100 columns. The array will initially contain zeros. The array will be named as embedding_matrix


---
Next, we will iterate through each word in our corpus by traversing the word_tokenizer.word_index dictionary that contains our words and their corresponding index.

Each word will be passed as key to the embedding_dictionary to retrieve the corresponding 100 dimensional vector for the word. The 100 dimensional vector will then be stored at the corresponding index of the word in the embedding_matrix



In [0]:
embedding_matrix = zeros((vocab_length, 100))
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

Our embedding_matrix now contains pretrained word embeddings for the words in our corpus.

In [0]:
model = Sequential()
embedding_layer = Embedding(vocab_length, 100, weights=[embedding_matrix], input_length=length_long_sentence, trainable=False)
model.add(embedding_layer)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))



*   The script remains the same, except for the embedding layer. Here in the embedding layer, the first parameter is the size of the vacabulary. The second parameter is the vector dimension of the output vector. Since we are using pretrained word embeddings that contain 100 dimensional vector, we set the vector dimension to 100.


*   Another very important attribute of the Embedding() layer that we did not use in the last section is weights. You can pass your pretrained embedding matrix as default weights to the weights parameter. And since we are not training the embedding layer, the trainable attribute has been set to False.



In [22]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 7, 100)            4400      
_________________________________________________________________
flatten_1 (Flatten)          (None, 700)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 701       
Total params: 5,101
Trainable params: 701
Non-trainable params: 4,400
_________________________________________________________________
None


We are again using adam as the optizer to minimize the loss. The loss function being used is binary_crossentropy. And we want to see the results in the form of accuracy so acc has been passed as the value for the metrics attribute.

In [24]:
model.fit(padded_sentence, sentiments, epochs=100, verbose=1)



Epoch 1/100
16/16 [==============================] - 0s 23ms/step - loss: 0.6466 - acc: 0.6250
Epoch 2/100
16/16 [==============================] - 0s 274us/step - loss: 0.6280 - acc: 0.6250
Epoch 3/100
16/16 [==============================] - 0s 289us/step - loss: 0.6075 - acc: 0.6250
Epoch 4/100
16/16 [==============================] - 0s 155us/step - loss: 0.5847 - acc: 0.6875
Epoch 5/100
16/16 [==============================] - 0s 122us/step - loss: 0.5643 - acc: 0.6875
Epoch 6/100
16/16 [==============================] - 0s 136us/step - loss: 0.5446 - acc: 0.6875
Epoch 7/100
16/16 [==============================] - 0s 116us/step - loss: 0.5258 - acc: 0.7500
Epoch 8/100
16/16 [==============================] - 0s 128us/step - loss: 0.5077 - acc: 0.7500
Epoch 9/100
16/16 [==============================] - 0s 113us/step - loss: 0.4905 - acc: 0.7500
Epoch 10/100
16/16 [==============================] - 0s 111us/step - loss: 0.4741 - acc: 0.7500
Epoch 11/100
16/16 [==================

In [25]:
loss, accuracy = model.evaluate(padded_sentence, sentiments, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 100.000000


### Word Embeddings with Keras Functions API



*   In this section, we will see how we can implement embedding layer with Keras Functional API.
*  The rest of the script remains similar as it was in the last section. The only change will be in the development of a deep learning model. Let's implement the same deep learning model as we implemented in the last section with Keras Functional API.



In [0]:
corpus = [
    # Positive Reviews

    'This is an excellent movie',
    'The move was fantastic I like it',
    'You should watch it is brilliant',
    'Exceptionally good',
    'Wonderfully directed and executed I like it',
    'Its a fantastic series',
    'Never watched such a brillent movie',
    'It is a Wonderful movie',

    # Negtive Reviews

    "horrible acting",
    'waste of money',
    'pathetic picture',
    'It was very boring',
    'I did not like the movie',
    'The movie was horrible',
    'I will not recommend',
    'The acting is pathetic'
]

In [0]:
sentiments = array([1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0])

In [0]:
from keras.preprocessing.text import Tokenizer

In [5]:
word_tokenizer = Tokenizer()
word_tokenizer.fit_on_texts(corpus)
corpus

['This is an excellent movie',
 'The move was fantastic I like it',
 'You should watch it is brilliant',
 'Exceptionally good',
 'Wonderfully directed and executed I like it',
 'Its a fantastic series',
 'Never watched such a brillent movie',
 'It is a Wonderful movie',
 'horrible acting',
 'waste of money',
 'pathetic picture',
 'It was very boring',
 'I did not like the movie',
 'The movie was horrible',
 'I will not recommend',
 'The acting is pathetic']

In [6]:
vocab_length = len(word_tokenizer.word_index) + 1
vocab_length

44

In [7]:
embedded_sentences = word_tokenizer.texts_to_sequences(corpus)
embedded_sentences

[[14, 3, 15, 16, 1],
 [4, 17, 6, 9, 5, 7, 2],
 [18, 19, 20, 2, 3, 21],
 [22, 23],
 [24, 25, 26, 27, 5, 7, 2],
 [28, 8, 9, 29],
 [30, 31, 32, 8, 33, 1],
 [2, 3, 8, 34, 1],
 [10, 11],
 [35, 36, 37],
 [12, 38],
 [2, 6, 39, 40],
 [5, 41, 13, 7, 4, 1],
 [4, 1, 6, 10],
 [5, 42, 13, 43],
 [4, 11, 3, 12]]

In [10]:
from nltk.tokenize import word_tokenize

word_count = lambda sentence: len(word_tokenize(sentence))
longest_sentence = max(corpus, key = word_count)
length_long_sentence = len(word_tokenize(longest_sentence))

padded_sentence = pad_sequences(embedded_sentences, length_long_sentence, padding='post')

padded_sentence

array([[14,  3, 15, 16,  1,  0,  0],
       [ 4, 17,  6,  9,  5,  7,  2],
       [18, 19, 20,  2,  3, 21,  0],
       [22, 23,  0,  0,  0,  0,  0],
       [24, 25, 26, 27,  5,  7,  2],
       [28,  8,  9, 29,  0,  0,  0],
       [30, 31, 32,  8, 33,  1,  0],
       [ 2,  3,  8, 34,  1,  0,  0],
       [10, 11,  0,  0,  0,  0,  0],
       [35, 36, 37,  0,  0,  0,  0],
       [12, 38,  0,  0,  0,  0,  0],
       [ 2,  6, 39, 40,  0,  0,  0],
       [ 5, 41, 13,  7,  4,  1,  0],
       [ 4,  1,  6, 10,  0,  0,  0],
       [ 5, 42, 13, 43,  0,  0,  0],
       [ 4, 11,  3, 12,  0,  0,  0]], dtype=int32)

In [0]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary =dict()
glove_file = open('/content/drive/My Drive/glove.6B.100d.txt', encoding='utf8')

In [0]:
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions

glove_file.close()

In [0]:
embedding_matrix = zeros((vocab_length, 100))
for word, index in word_tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [0]:
from keras.models import Model
from keras.layers import Input

deep_inputs = Input(shape=(length_long_sentence,))
embedding = Embedding(vocab_length, 100, weights=[embedding_matrix], input_length=length_long_sentence, trainable=False)(deep_inputs) # line A
flatten = Flatten()(embedding)
hidden = Dense(1, activation='sigmoid')(flatten)
model = Model(inputs=deep_inputs, outputs=hidden)



*   In the Keras Functional API, you have to define the input layer separately before the embedding layer. In the input, layer you have to simply pass the length of input vector. To specify that previous layer as input to the next layer, the previous layer is passed as a parameter inside the parenthesis, at the end of the next layer.


*   For instance, in the above script, you can see that deep_inputs is passed as parameter at the end of the embedding layer. Similarly, embedding is passed as input at the end of the Flatten() layer and so on.

Finally, in the Model(), you have to pass the input layer, and the final output layer.



In [20]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['acc'])
print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 7)                 0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 7, 100)            4400      
_________________________________________________________________
flatten_1 (Flatten)          (None, 700)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 701       
Total params: 5,101
Trainable params: 701
Non-trainable params: 4,400
_________________________________________________________________
None


In [21]:
model.fit(padded_sentence, sentiments, epochs=100, verbose=1)
loss, accuracy = model.evaluate(padded_sentence, sentiments,verbose=0)
print('Accuracy: %f' % (accuracy*100))



Epoch 1/100
16/16 [==============================] - 0s 24ms/step - loss: 0.7862 - acc: 0.3125
Epoch 2/100
16/16 [==============================] - 0s 142us/step - loss: 0.7563 - acc: 0.3125
Epoch 3/100
16/16 [==============================] - 0s 132us/step - loss: 0.7277 - acc: 0.3750
Epoch 4/100
16/16 [==============================] - 0s 290us/step - loss: 0.7002 - acc: 0.4375
Epoch 5/100
16/16 [==============================] - 0s 127us/step - loss: 0.6740 - acc: 0.5625
Epoch 6/100
16/16 [==============================] - 0s 311us/step - loss: 0.6490 - acc: 0.6250
Epoch 7/100
16/16 [==============================] - 0s 137us/step - loss: 0.6252 - acc: 0.7500
Epoch 8/100
16/16 [==============================] - 0s 177us/step - loss: 0.6025 - acc: 0.7500
Epoch 9/100
16/16 [==============================] - 0s 141us/step - loss: 0.5810 - acc: 0.7500
Epoch 10/100
16/16 [==============================] - 0s 163us/step - loss: 0.5604 - acc: 0.7500
Epoch 11/100
16/16 [==================